In [1]:
import sympy
from sympy import *
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
plt.style.use("seaborn")
import pandas as pd
import cufflinks as cf
import pyfolio as pyf
import plotly.express as px
from itertools import product

/Users/miguelbetances/opt/anaconda3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning:

Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.



# Get Data

In [2]:
df = yf.download('NQ=F', period = '3mo', interval = '60m') #start = '2022-07-13', end = '2023-09-12')
df.to_csv('stocks')
df.rename(columns = {'Adj Close':'AdjClose'}, inplace = True)

[*********************100%***********************]  1 of 1 completed


# Arrange Dataframe

In [5]:
def stock_calculations(data, col, p1, p2, roc_window):
    short_ema = data[col].ewm(span = p1, min_periods = p1, adjust=False).mean()
    long_sma = data[col].rolling(window = p2).mean()
    data['short_ema'] = short_ema
    data['long_sma'] = long_sma
    data['difference'] = (short_ema - long_sma)
    data['difference_ma'] = data['difference'].rolling(window=25).mean()
    data['deriv'] = diff(data['difference_ma'])
    data['deriv2'] = diff(data['deriv'])
    data['roc'] = data[col].pct_change(periods=roc_window)*100
    data['z'] = data['difference'] * .1

In [6]:
stock_calculations(df, 'AdjClose', 50, 200, 60)

In [26]:
derivative_data = df.dropna()
derivative_data = derivative_data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis =1)
derivative_data.head()

,AdjClose,short_ema,long_sma,difference,difference_ma,deriv,deriv2,roc,z
Datetime,,,,,,,,,
2023-11-20 10:00:00-05:00,15999.75,15884.876559,15645.86750,239.009059,259.967921,-2.296044,0.150717,1.046798,23.900906
2023-11-20 11:00:00-05:00,16013.25,15889.910812,15649.11000,240.800812,257.859591,-2.108330,0.187713,1.143236,24.080081
2023-11-20 12:00:00-05:00,16010.75,15894.649604,15652.33500,242.314604,255.946216,-1.913375,0.194955,1.095518,24.231460
2023-11-20 13:00:00-05:00,16070.75,15901.555501,15655.81875,245.736751,254.280878,-1.665338,0.248037,1.389546,24.573675
2023-11-20 14:00:00-05:00,16094.50,15909.121952,15659.52375,249.598202,252.862531,-1.418347,0.246991,1.476963,24.959820


In [9]:
derivative_data['deriv2'].mean()

0.002932966418773302

# Define Strat

In [10]:
alpha = derivative_data['deriv'].mean() *.1
alpha2 = derivative_data['deriv2'].mean() 
count = 0
position_status = {'Type':None, 'EntryPrice':None, 'Date':None}
condition_met = {'D1': None, 'D2': None, 'ClosePrice': None, 'Rate': None}
profits = 0
positions = []
realized_profits = []
longs = []
shorts = []
pos = []
for row in derivative_data.iterrows():
    positions.append(position_status['Type'])
    realized_profits.append(profits)
    close_price = row[1][0]
    long_ma = row[1][2]
    rate = row[1][7]
    d1 = row[1][5]
    d2 = row[1][6]
    difference_value = row[1][4]
    position_date = row[0]
    if d1 < alpha and d2 > 0 and close_price < long_ma and rate > 0 and position_status['Type'] != 'Long':
        if position_status['Type'] == 'Short':
            profit = (close_price-position_status['EntryPrice'])*-1
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(1)
        if position_status['Type'] == None:
            pos.append(1)
            position_status['Type'] = 'Long'
            position_status['EntryPrice'] = close_price
            position_status['Date'] = position_date
    elif d1 > alpha and d2 < 0 and close_price > long_ma and rate < 0 and position_status['Type'] != 'Short':
        if position_status['Type'] == 'Long':
            profit = (close_price-position_status['EntryPrice'])
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(-1)
        if position_status['Type'] == None:
            pos.append(-1)
            position_status['Type'] = 'Short'
            position_status['EntryPrice'] = close_price
            position_status['Date'] = position_date
    elif d2 < (alpha2*-1) and rate < 0:
        if position_status['Type'] == 'Long':
            profit = (close_price-position_status['EntryPrice'])
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(-1)
    elif d2 > alpha2 and rate > 0:
        if position_status['Type'] == 'Short':
            profit = (close_price-position_status['EntryPrice'])*-1
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(1)
    count += 1
    if len(realized_profits) < count:
        realized_profits.append(0)
    if len(pos) < count:
        pos.append(0)
derivative_data['realized_profits'] = realized_profits
derivative_data['pos'] = pos

In [11]:
values = derivative_data[derivative_data['pos'] == 1]
print(len(values))

5


# Marking entry and exit

In [12]:
entry = [derivative_data.loc[(derivative_data.pos == 1)]][0]
exit = [derivative_data.loc[(derivative_data.pos == -1)]][0]

In [13]:
exit['exit'] = exit['AdjClose']
exit = exit.drop('AdjClose', axis = 1)

In [14]:
entry['entry'] = entry['AdjClose']
entry= entry.drop('AdjClose', axis = 1)

In [15]:
entry = pd.DataFrame(index = entry.index, data = {'entry':entry.entry})

In [16]:
len(entry)

5

In [17]:
exit = pd.DataFrame(index = exit.index, data = {'exit':exit.exit})

In [18]:
len(exit)

6

In [19]:
derivative_data = derivative_data.join(entry)

In [20]:
derivative_data = derivative_data.join(exit)

In [21]:
buy_hold_return = (derivative_data.AdjClose - derivative_data.AdjClose.shift(1)).cumsum()
derivative_data['bnh'] = buy_hold_return

In [22]:
derivative_data

,AdjClose,short_ema,long_sma,difference,difference_ma,deriv,deriv2,roc,z,realized_profits,pos,entry,exit,bnh
Datetime,,,,,,,,,,,,,,
2023-11-20 10:00:00-05:00,15999.75,15884.876559,15645.86750,239.009059,259.967921,-2.296044,0.150717,1.046798,23.900906,0.00,0,NaN,NaN,NaN
2023-11-20 11:00:00-05:00,16013.25,15889.910812,15649.11000,240.800812,257.859591,-2.108330,0.187713,1.143236,24.080081,0.00,0,NaN,NaN,13.50
2023-11-20 12:00:00-05:00,16010.75,15894.649604,15652.33500,242.314604,255.946216,-1.913375,0.194955,1.095518,24.231460,0.00,0,NaN,NaN,11.00
2023-11-20 13:00:00-05:00,16070.75,15901.555501,15655.81875,245.736751,254.280878,-1.665338,0.248037,1.389546,24.573675,0.00,0,NaN,NaN,71.00
2023-11-20 14:00:00-05:00,16094.50,15909.121952,15659.52375,249.598202,252.862531,-1.418347,0.246991,1.476963,24.959820,0.00,0,NaN,NaN,94.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-06 12:00:00-05:00,17591.50,17668.163809,17571.02250,97.141309,85.263964,1.676255,-0.075625,-0.063910,9.714131,660.75,0,NaN,NaN,1591.75
2024-02-06 13:00:00-05:00,17630.75,17666.696601,17571.16875,95.527851,86.827035,1.563071,-0.113184,0.141999,9.552785,660.75,0,NaN,NaN,1631.00
2024-02-06 14:00:00-05:00,17635.50,17665.473205,17571.31875,94.154455,88.241323,1.414288,-0.148783,0.133432,9.415445,660.75,0,NaN,NaN,1635.75


# Plots

In [23]:
fig = px.line(derivative_data, x=derivative_data.index, y=['AdjClose','short_ema', 'long_sma','difference'])
fig.add_scatter(x=derivative_data.index, y=derivative_data.entry, mode="markers", marker=dict(size=8,color="yellow"), name = 'Entry')
fig.add_scatter(x=derivative_data.index, y=derivative_data.exit, mode="markers", marker=dict(size=8,color="Red"), name = 'Exit')
fig.show()

In [24]:
fig2 = px.line(derivative_data, x=derivative_data.index, y=['z', 'deriv', 'deriv2', 'roc'])
fig2.show()

In [25]:
derivative_data[['bnh', 'realized_profits']].iplot()